# Import the packages

In [4]:
import pandas as pd
import numpy as np
import ujson as json
import torch
from transformers import BertTokenizer

# Load/Prepare Datasets

In [30]:
# Load the twitter DS

# df = pd.read_csv(PATH['twitter'], names = ['tweet', 'hate_speech'])

# 24784
df = pd.read_csv('datasets/twitter.csv') # https://github.com/t-davidson/hate-speech-and-offensive-language
# print(df.head(10))

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tweets = df.loc[:,'tweet'].values
labels = df.loc[:,'hate_speech'].values

ids = []
atn_mask = []
# encoded = []
for tweet in tweets:
    encoded = tokenizer.encode_plus(tweet, add_special_tokens=True)
    ids.append(encoded['input_ids'])

    print(tweet)
    print(tokenizer.tokenize(tweet))
    print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet)))
    break
    
# print(len(df_tokenized)) 
# print(tweets[:10])
# print(ids[:10])



!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
['!', '!', '!', 'rt', '@', 'maya', '##sol', '##ove', '##ly', ':', 'as', 'a', 'woman', 'you', 'shouldn', "'", 't', 'complain', 'about', 'cleaning', 'up', 'your', 'house', '.', '&', 'amp', ';', 'as', 'a', 'man', 'you', 'should', 'always', 'take', 'the', 'trash', 'out', '.', '.', '.']
[999, 999, 999, 19387, 1030, 9815, 19454, 21818, 2135, 1024, 2004, 1037, 2450, 2017, 5807, 1005, 1056, 17612, 2055, 9344, 2039, 2115, 2160, 1012, 1004, 23713, 1025, 2004, 1037, 2158, 2017, 2323, 2467, 2202, 1996, 11669, 2041, 1012, 1012, 1012]


In [12]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
parler = map(json.loads, open('./datasets/parler_pretrain.ndjson'))
parler = pd.DataFrame.from_records(parler)
parler = parler.loc[(parler['body']!='')] # Filter empty body. Final len: 636420 outof 1094836 (58.12925406179556%)
parler = parler[['body']]
print(len(parler))
parler.head(100)

KeyboardInterrupt: 

,tweet,offensive_language
0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,3
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,3
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,2
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,6
5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",2
6,"!!!!!!""@__BrighterDays: I can not just sit up ...",3
7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,3
8,""" &amp; you might not get ya bitch back &amp; ...",3
9,""" @rhythmixx_ :hobbies include: fighting Maria...",2


In [ ]:


reddit.head()
gab.head()

NameError: name 'reddit' is not defined

In [ ]:
reddit = pd.read_csv('reddit.csv') # https://github.com/jing-qian/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech/tree/master/data


In [ ]:
gab = pd.read_csv('gab.csv')

# Build Data Preprocessing Pipeline

In [ ]:
# pip install transformers
from transformers import BertTokenizer
twitter.loc[offnsive_language != 0].sam'ple(10)['tweet']

ModuleNotFoundError: No module named 'transformers'